In [24]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import json
train_idx = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camId = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelist = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idx = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat3_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idx = loadmat('PR_data/cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()
with open('PR_data/feature_data.json', 'r') as f:
    features = np.array(json.load(f))
train_idx -= 1

In [29]:
train_features = features[train_idx.tolist()]
gallery_features = features[gallery_idx.tolist()]
query_features = features[query_idx.tolist()]


In [33]:
query_features.shape, gallery_features.shape

((1400, 2048), (5328, 2048))

(1, 1, 3, 3)